In [81]:
import pandas as pd
import json
import glob
import re

In [82]:
def simplify_text(s):
    s1 = s.split(".")[-1]
    s1 = re.sub(r"\W+", " ", s1)
    s1 = " ".join(s1.split())
    s1 = s1.lower()
    return s1

In [83]:
# bs_all_accs = set()
report_types_bs_is = {"CDKT":{},
                        "KQKD":{}
                     }
report_types_cf = {"LC": set()
                    }
lookup_dict_all = {"CDKT":{},
                        "KQKD":{}
                     }

In [84]:
for report_type, report_all_accounts in report_types_bs_is.items():
    lookup_dict = {}
    
    for file in glob.glob(f'functions/localData/financeInfo/*{report_type}_Annual*.json'):
        with open(file, 'r') as jsonfile:
            try:
                j = json.load(jsonfile)
                for report_fullname, report_content in j[1].items():
                    for acc in report_content:
                        acc_id = acc['ReportNormID']
                        if acc_id not in lookup_dict.keys():
                            lookup_dict[acc_id] = acc
            except:
                print("EXCEPTION IN LOOKUP DICT")
    
    lookup_dict_all[report_type] = lookup_dict
    
    for acc_id, acc in lookup_dict.items():
        acc_n = simplify_text(acc['NameEn'])
#         acc_n = acc['NameEn']
        parent_n = simplify_text(lookup_dict[acc['ParentReportNormID']]['NameEn'])
#         parent_n = lookup_dict[acc['ParentReportNormID']]['NameEn']
        acc_new_key = f'{acc_n};{parent_n}'

        if acc_new_key not in report_all_accounts.keys():
            report_all_accounts[acc_new_key] = [acc_id]
        else:
            if acc_id not in report_all_accounts[acc_new_key]:
                report_all_accounts[acc_new_key].append(acc_id)
                        
#     report_all_accounts = dict(sorted(report_all_accounts.items()))
    with open(f'functions/schema/{report_type}_all_accounts.json', 'w') as writefile:
        json.dump(report_all_accounts, writefile, ensure_ascii=False, indent=4)

In [85]:
ids = []
keys = []
max_len_v = 0
for k, v in report_types_bs_is['CDKT'].items():
    ids = ids + v
    keys.append(k)
    if len(v) > max_len_v:
        max_len_v = len(v)
    if len(v) == 5:
        for i in v:
            print(lookup_dict_all['CDKT'][i])

print(len(ids))
print(len(set(ids)))
print(len(keys))
print(len(set(keys)))
print(max_len_v)

{'ID': 67, 'ReportNormID': 2994, 'Name': 'NGUỒN VỐN', 'NameEn': "OWNER'S EQUITY", 'NameMobile': 'NGUỒN VỐN', 'NameMobileEn': "OWNER'S EQUITY", 'CssStyle': 'MaxB', 'Padding': 'Padding1', 'ParentReportNormID': 2994, 'ReportComponentName': 'Cân đối kế toán', 'ReportComponentNameEn': 'Balance Sheet', 'Unit': None, 'UnitEn': None, 'OrderType': None, 'OrderingComponent': None, 'RowNumber': None, 'ReportComponentTypeID': None, 'ChildTotal': 0, 'Levels': 0, 'Value1': None, 'Value2': None, 'Value3': None, 'Value4': None, 'Vl': None, 'IsShowData': True}
{'ID': 100, 'ReportNormID': 3015, 'Name': 'I. Vốn chủ sở hữu', 'NameEn': "I. Owner's equity", 'NameMobile': 'I. Vốn chủ sở hữu', 'NameMobileEn': "I. Owner's equity", 'CssStyle': 'NormalB', 'Padding': 'Padding2', 'ParentReportNormID': 2998, 'ReportComponentName': 'Cân đối kế toán', 'ReportComponentNameEn': 'Balance Sheet', 'Unit': None, 'UnitEn': None, 'OrderType': None, 'OrderingComponent': None, 'RowNumber': None, 'ReportComponentTypeID': None, 